# Data Exploration for Harmonia - Emotional Music Analysis

This notebook performs exploratory data analysis (EDA) to understand the relationship between musical features and emotional dimensions using the Circumplex Model (Valence-Arousal).

## Objectives:
1. Analyze emotional label distributions in our dataset
2. Extract and explore musical features using librosa
3. Identify correlations between audio features and emotional dimensions
4. Visualize the emotional landscape of our music dataset

In [ ]:
# Import Required Libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import librosa
import librosa.display
import warnings
from pathlib import Path
import os

# Set style for better visualizations
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")
warnings.filterwarnings('ignore')

# Set random seed for reproducibility
np.random.seed(42)

## 1. Data Loading and Initial Inspection

Let's start by loading our dataset and examining the basic structure.

In [ ]:
# Define data paths
DATA_DIR = Path('../data')
RAW_DIR = DATA_DIR / 'raw'
PROCESSED_DIR = DATA_DIR / 'processed'
DATASET_DIR = Path('../dataset')

# Check if dataset directories exist
print(f"Raw data directory exists: {RAW_DIR.exists()}")
print(f"Processed data directory exists: {PROCESSED_DIR.exists()}")
print(f"Dataset directory exists: {DATASET_DIR.exists()}")

# List available datasets
if DATASET_DIR.exists():
    print("\nAvailable datasets:")
    for item in DATASET_DIR.iterdir():
        if item.is_dir():
            print(f"- {item.name}")
            for subitem in item.iterdir():
                print(f"  └── {subitem.name}")

In [ ]:
# Function to load DEAM dataset annotations
def load_deam_annotations(dataset_path):
    """
    Load DEAM dataset annotations from the extracted files
    """
    deam_path = dataset_path / 'DEAM'
    
    # Check if DEAM annotations exist
    annotations_files = list(deam_path.glob('*annotations*'))
    if annotations_files:
        print(f"Found annotation files: {[f.name for f in annotations_files]}")
        # TODO: Implement actual loading based on file structure
        return None
    else:
        print("No annotation files found. Please extract the DEAM_Annotations.zip file.")
        return None

# Attempt to load dataset
annotations_df = load_deam_annotations(DATASET_DIR)

# For demonstration, create a sample dataset structure
# This will be replaced with actual data loading once datasets are extracted
print("\nCreating sample data structure for demonstration...")
sample_data = {
    'song_id': [f'song_{i:03d}' for i in range(100)],
    'valence': np.random.normal(0, 0.3, 100),  # Centered around neutral
    'arousal': np.random.normal(0, 0.4, 100),  # Slightly more varied
    'file_path': [f'../dataset/DEAM/audio/song_{i:03d}.mp3' for i in range(100)]
}

# Clip values to realistic range [-1, 1]
sample_data['valence'] = np.clip(sample_data['valence'], -1, 1)
sample_data['arousal'] = np.clip(sample_data['arousal'], -1, 1)

df = pd.DataFrame(sample_data)
print(f"\nSample dataset shape: {df.shape}")
print("\nFirst 5 rows:")
df.head()

## 2. Emotional Label Distribution Analysis

Understanding the distribution of emotional labels in our dataset is crucial for model training.

In [ ]:
# Basic statistics for emotional dimensions
print("Emotional Dimensions - Descriptive Statistics:")
print(df[['valence', 'arousal']].describe())

# Check for missing values
print("\nMissing values:")
print(df[['valence', 'arousal']].isnull().sum())

In [ ]:
# Visualize emotional distributions
fig, axes = plt.subplots(2, 2, figsize=(15, 10))

# Valence histogram
axes[0,0].hist(df['valence'], bins=20, alpha=0.7, color='skyblue', edgecolor='black')
axes[0,0].set_title('Valence Distribution', fontsize=14, fontweight='bold')
axes[0,0].set_xlabel('Valence (Pleasure)')
axes[0,0].set_ylabel('Frequency')
axes[0,0].axvline(df['valence'].mean(), color='red', linestyle='--', label=f'Mean: {df["valence"].mean():.3f}')
axes[0,0].legend()

# Arousal histogram
axes[0,1].hist(df['arousal'], bins=20, alpha=0.7, color='lightcoral', edgecolor='black')
axes[0,1].set_title('Arousal Distribution', fontsize=14, fontweight='bold')
axes[0,1].set_xlabel('Arousal (Activation)')
axes[0,1].set_ylabel('Frequency')
axes[0,1].axvline(df['arousal'].mean(), color='red', linestyle='--', label=f'Mean: {df["arousal"].mean():.3f}')
axes[0,1].legend()

# The Circumplex Model - 2D Emotional Space
scatter = axes[1,0].scatter(df['valence'], df['arousal'], alpha=0.6, s=50, c=range(len(df)), cmap='viridis')
axes[1,0].set_title('Circumplex Model: Emotional Landscape', fontsize=14, fontweight='bold')
axes[1,0].set_xlabel('Valence (Negative ← → Positive)')
axes[1,0].set_ylabel('Arousal (Low ← → High)')
axes[1,0].grid(True, alpha=0.3)
axes[1,0].axhline(y=0, color='black', linestyle='-', alpha=0.3)
axes[1,0].axvline(x=0, color='black', linestyle='-', alpha=0.3)

# Add quadrant labels
axes[1,0].text(0.5, 0.5, 'High Arousal\nPositive Valence\n(Joy, Excitement)', 
               transform=axes[1,0].transAxes, ha='center', va='center', 
               bbox=dict(boxstyle='round', facecolor='yellow', alpha=0.3))
axes[1,0].text(0.5, -0.1, 'Low Arousal\nPositive Valence\n(Serenity, Contentment)', 
               transform=axes[1,0].transAxes, ha='center', va='center',
               bbox=dict(boxstyle='round', facecolor='lightgreen', alpha=0.3))
axes[1,0].text(-0.1, 0.5, 'High Arousal\nNegative Valence\n(Anger, Fear)', 
               transform=axes[1,0].transAxes, ha='center', va='center',
               bbox=dict(boxstyle='round', facecolor='red', alpha=0.3))
axes[1,0].text(-0.1, -0.1, 'Low Arousal\nNegative Valence\n(Sadness, Depression)', 
               transform=axes[1,0].transAxes, ha='center', va='center',
               bbox=dict(boxstyle='round', facecolor='lightblue', alpha=0.3))

# Correlation between Valence and Arousal
correlation = df['valence'].corr(df['arousal'])
axes[1,1].scatter(df['valence'], df['arousal'], alpha=0.6)
axes[1,1].set_title(f'Valence vs Arousal Correlation\n(r = {correlation:.3f})', fontsize=14, fontweight='bold')
axes[1,1].set_xlabel('Valence')
axes[1,1].set_ylabel('Arousal')
axes[1,1].grid(True, alpha=0.3)

# Add trend line
z = np.polyfit(df['valence'], df['arousal'], 1)
p = np.poly1d(z)
axes[1,1].plot(df['valence'], p(df['valence']), "r--", alpha=0.8)

plt.tight_layout()
plt.show()

## 3. Audio Feature Extraction Framework

Now we'll set up the framework for extracting musical features using librosa.

In [ ]:
def extract_audio_features(file_path, sr=22050, duration=30):
    """
    Extract comprehensive audio features from a music file
    
    Parameters:
    - file_path: path to audio file
    - sr: sample rate
    - duration: duration in seconds to analyze (None for full track)
    
    Returns:
    - Dictionary of extracted features
    """
    try:
        # Load audio file
        y, sr = librosa.load(file_path, sr=sr, duration=duration)
        
        # Initialize feature dictionary
        features = {}
        
        # 1. Rhythm/Tempo Features
        tempo, beats = librosa.beat.beat_track(y=y, sr=sr)
        features['tempo'] = tempo
        features['beat_density'] = len(beats) / (len(y) / sr)  # beats per second
        
        # 2. Spectral Features (Timbre/Texture)
        # MFCCs (Mel-Frequency Cepstral Coefficients)
        mfccs = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=20)
        for i in range(20):
            features[f'mfcc_{i+1}_mean'] = np.mean(mfccs[i])
            features[f'mfcc_{i+1}_std'] = np.std(mfccs[i])
        
        # Spectral centroid (brightness)
        spectral_centroids = librosa.feature.spectral_centroid(y=y, sr=sr)[0]
        features['spectral_centroid_mean'] = np.mean(spectral_centroids)
        features['spectral_centroid_std'] = np.std(spectral_centroids)
        
        # Spectral bandwidth
        spectral_bandwidth = librosa.feature.spectral_bandwidth(y=y, sr=sr)[0]
        features['spectral_bandwidth_mean'] = np.mean(spectral_bandwidth)
        features['spectral_bandwidth_std'] = np.std(spectral_bandwidth)
        
        # Spectral rolloff
        spectral_rolloff = librosa.feature.spectral_rolloff(y=y, sr=sr)[0]
        features['spectral_rolloff_mean'] = np.mean(spectral_rolloff)
        features['spectral_rolloff_std'] = np.std(spectral_rolloff)
        
        # Zero crossing rate
        zcr = librosa.feature.zero_crossing_rate(y)[0]
        features['zcr_mean'] = np.mean(zcr)
        features['zcr_std'] = np.std(zcr)
        
        # 3. Pitch/Harmony Features
        # Chroma features
        chroma = librosa.feature.chroma_stft(y=y, sr=sr)
        for i in range(12):
            features[f'chroma_{i+1}_mean'] = np.mean(chroma[i])
            features[f'chroma_{i+1}_std'] = np.std(chroma[i])
        
        # Tonnetz (Tonal centroid features)
        tonnetz = librosa.feature.tonnetz(y=librosa.effects.harmonic(y), sr=sr)
        for i in range(6):
            features[f'tonnetz_{i+1}_mean'] = np.mean(tonnetz[i])
            features[f'tonnetz_{i+1}_std'] = np.std(tonnetz[i])
        
        # 4. Energy Features
        # RMS Energy
        rms = librosa.feature.rms(y=y)[0]
        features['rms_mean'] = np.mean(rms)
        features['rms_std'] = np.std(rms)
        
        return features
        
    except Exception as e:
        print(f"Error processing {file_path}: {str(e)}")
        return None

# Demonstrate feature extraction with a synthetic audio signal
print("Demonstrating feature extraction with synthetic audio...")

# Create a synthetic audio signal for demonstration
sr = 22050
duration = 5  # 5 seconds
t = np.linspace(0, duration, sr * duration)

# Create a simple melody with varying frequencies
frequencies = [440, 523, 659, 784]  # A4, C5, E5, G5
synthetic_audio = np.concatenate([
    np.sin(2 * np.pi * freq * t[:sr]) for freq in frequencies
])

# Add some noise for realism
synthetic_audio += 0.1 * np.random.normal(0, 1, len(synthetic_audio))

# Normalize
synthetic_audio = synthetic_audio / np.max(np.abs(synthetic_audio))

print(f"Created synthetic audio signal: {len(synthetic_audio)} samples, {len(synthetic_audio)/sr:.1f} seconds")

In [ ]:
# Visualize the synthetic audio and its features
fig, axes = plt.subplots(2, 2, figsize=(15, 10))

# Time domain plot
time_axis = np.linspace(0, len(synthetic_audio)/sr, len(synthetic_audio))
axes[0,0].plot(time_axis, synthetic_audio)
axes[0,0].set_title('Synthetic Audio Signal (Time Domain)')
axes[0,0].set_xlabel('Time (seconds)')
axes[0,0].set_ylabel('Amplitude')

# Spectrogram
D = librosa.stft(synthetic_audio)
S_db = librosa.amplitude_to_db(np.abs(D), ref=np.max)
librosa.display.specshow(S_db, sr=sr, x_axis='time', y_axis='hz', ax=axes[0,1])
axes[0,1].set_title('Spectrogram')
axes[0,1].set_ylabel('Frequency (Hz)')

# MFCC features
mfccs = librosa.feature.mfcc(y=synthetic_audio, sr=sr, n_mfcc=13)
librosa.display.specshow(mfccs, sr=sr, x_axis='time', ax=axes[1,0])
axes[1,0].set_title('MFCC Features')
axes[1,0].set_ylabel('MFCC Coefficients')

# Chroma features
chroma = librosa.feature.chroma_stft(y=synthetic_audio, sr=sr)
librosa.display.specshow(chroma, sr=sr, x_axis='time', y_axis='chroma', ax=axes[1,1])
axes[1,1].set_title('Chroma Features')
axes[1,1].set_ylabel('Pitch Class')

plt.tight_layout()
plt.show()

# Extract features from synthetic audio (simulate file processing)
print("\nExtracting features from synthetic audio...")
# We'll simulate this since we don't have an actual file
sample_features = {
    'tempo': 120.0,
    'spectral_centroid_mean': 2000.5,
    'mfcc_1_mean': -150.2,
    'chroma_1_mean': 0.7,
    'rms_mean': 0.15
}

print("Sample extracted features:")
for feature, value in sample_features.items():
    print(f"{feature}: {value:.3f}")

## 4. Feature-Emotion Correlation Analysis

This section will analyze the relationships between extracted audio features and emotional dimensions.

In [ ]:
# For demonstration, let's create some synthetic feature data that correlates with emotions
np.random.seed(42)

# Create synthetic audio features for our sample dataset
n_samples = len(df)

# Generate features with some correlation to valence and arousal
audio_features = {
    # Tempo often correlates with arousal
    'tempo': 100 + 40 * df['arousal'] + np.random.normal(0, 15, n_samples),
    
    # Spectral centroid (brightness) often correlates with valence
    'spectral_centroid_mean': 2000 + 500 * df['valence'] + np.random.normal(0, 200, n_samples),
    
    # RMS energy often correlates with arousal
    'rms_mean': 0.1 + 0.05 * df['arousal'] + np.random.normal(0, 0.02, n_samples),
    
    # Add some uncorrelated features for comparison
    'mfcc_1_mean': np.random.normal(-150, 20, n_samples),
    'mfcc_2_mean': np.random.normal(50, 15, n_samples),
    
    # Zero crossing rate might correlate with arousal (percussive content)
    'zcr_mean': 0.05 + 0.02 * df['arousal'] + np.random.normal(0, 0.01, n_samples),
    
    # Spectral bandwidth
    'spectral_bandwidth_mean': 1000 + 200 * df['valence'] + 150 * df['arousal'] + np.random.normal(0, 100, n_samples),
}

# Add audio features to dataframe
for feature, values in audio_features.items():
    df[feature] = values

print(f"Extended dataset shape: {df.shape}")
print("\nAudio features added:")
for feature in audio_features.keys():
    print(f"- {feature}")

In [ ]:
# Calculate correlation matrix
feature_columns = list(audio_features.keys())
emotion_columns = ['valence', 'arousal']
all_columns = emotion_columns + feature_columns

correlation_matrix = df[all_columns].corr()

# Create correlation heatmap
plt.figure(figsize=(12, 10))
mask = np.triu(np.ones_like(correlation_matrix, dtype=bool))
sns.heatmap(correlation_matrix, 
            mask=mask,
            annot=True, 
            cmap='RdBu_r', 
            center=0,
            square=True,
            fmt='.3f',
            cbar_kws={"shrink": .8})
plt.title('Feature-Emotion Correlation Matrix', fontsize=16, fontweight='bold')
plt.tight_layout()
plt.show()

# Extract correlations with emotions specifically
valence_correlations = correlation_matrix['valence'].drop('valence').sort_values(key=abs, ascending=False)
arousal_correlations = correlation_matrix['arousal'].drop('arousal').sort_values(key=abs, ascending=False)

print("\nStrongest correlations with VALENCE:")
for feature, corr in valence_correlations.head(5).items():
    print(f"{feature}: {corr:.3f}")

print("\nStrongest correlations with AROUSAL:")
for feature, corr in arousal_correlations.head(5).items():
    print(f"{feature}: {corr:.3f}")

In [ ]:
# Create detailed scatter plots for the most correlated features
fig, axes = plt.subplots(2, 3, figsize=(18, 12))

# Top valence correlations
top_valence_features = valence_correlations.head(3)
for i, (feature, corr) in enumerate(top_valence_features.items()):
    axes[0, i].scatter(df[feature], df['valence'], alpha=0.6, s=50)
    axes[0, i].set_xlabel(feature)
    axes[0, i].set_ylabel('Valence')
    axes[0, i].set_title(f'{feature} vs Valence\n(r = {corr:.3f})')
    
    # Add trend line
    z = np.polyfit(df[feature], df['valence'], 1)
    p = np.poly1d(z)
    axes[0, i].plot(df[feature], p(df[feature]), "r--", alpha=0.8)
    axes[0, i].grid(True, alpha=0.3)

# Top arousal correlations
top_arousal_features = arousal_correlations.head(3)
for i, (feature, corr) in enumerate(top_arousal_features.items()):
    axes[1, i].scatter(df[feature], df['arousal'], alpha=0.6, s=50, color='orange')
    axes[1, i].set_xlabel(feature)
    axes[1, i].set_ylabel('Arousal')
    axes[1, i].set_title(f'{feature} vs Arousal\n(r = {corr:.3f})')
    
    # Add trend line
    z = np.polyfit(df[feature], df['arousal'], 1)
    p = np.poly1d(z)
    axes[1, i].plot(df[feature], p(df[feature]), "r--", alpha=0.8)
    axes[1, i].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## 5. Key Insights and Hypotheses

Based on our exploratory analysis, let's formulate key insights and hypotheses for model development.

In [ ]:
# Statistical significance testing for correlations
from scipy.stats import pearsonr

print("STATISTICAL SIGNIFICANCE OF CORRELATIONS")
print("=" * 50)

print("\nVALENCE CORRELATIONS:")
for feature in valence_correlations.head(5).index:
    corr, p_value = pearsonr(df[feature], df['valence'])
    significance = "***" if p_value < 0.001 else "**" if p_value < 0.01 else "*" if p_value < 0.05 else "ns"
    print(f"{feature}: r = {corr:.3f}, p = {p_value:.4f} {significance}")

print("\nAROUSAL CORRELATIONS:")
for feature in arousal_correlations.head(5).index:
    corr, p_value = pearsonr(df[feature], df['arousal'])
    significance = "***" if p_value < 0.001 else "**" if p_value < 0.01 else "*" if p_value < 0.05 else "ns"
    print(f"{feature}: r = {corr:.3f}, p = {p_value:.4f} {significance}")

print("\nLegend: *** p<0.001, ** p<0.01, * p<0.05, ns = not significant")

In [ ]:
# Feature importance ranking for model development
feature_importance = pd.DataFrame({
    'feature': feature_columns,
    'valence_corr': [correlation_matrix.loc[feature, 'valence'] for feature in feature_columns],
    'arousal_corr': [correlation_matrix.loc[feature, 'arousal'] for feature in feature_columns],
})

# Calculate combined importance score
feature_importance['combined_importance'] = (
    abs(feature_importance['valence_corr']) + abs(feature_importance['arousal_corr'])
)

feature_importance = feature_importance.sort_values('combined_importance', ascending=False)

print("FEATURE IMPORTANCE RANKING FOR MODEL DEVELOPMENT")
print("=" * 60)
print(f"{'Feature':<25} {'Valence':<10} {'Arousal':<10} {'Combined':<10}")
print("-" * 60)

for _, row in feature_importance.iterrows():
    print(f"{row['feature']:<25} {row['valence_corr']:>8.3f} {row['arousal_corr']:>9.3f} {row['combined_importance']:>9.3f}")

## 6. Next Steps and Recommendations

Based on this exploratory analysis, here are the key findings and recommendations for the next phases:

In [ ]:
# Generate summary statistics and save processed data
summary_stats = {
    'dataset_size': len(df),
    'valence_range': (df['valence'].min(), df['valence'].max()),
    'arousal_range': (df['arousal'].min(), df['arousal'].max()),
    'valence_mean': df['valence'].mean(),
    'arousal_mean': df['arousal'].mean(),
    'valence_arousal_correlation': df['valence'].corr(df['arousal']),
    'top_valence_predictors': valence_correlations.head(3).to_dict(),
    'top_arousal_predictors': arousal_correlations.head(3).to_dict(),
}

print("DATASET SUMMARY")
print("=" * 40)
print(f"Total samples: {summary_stats['dataset_size']}")
print(f"Valence range: {summary_stats['valence_range'][0]:.3f} to {summary_stats['valence_range'][1]:.3f}")
print(f"Arousal range: {summary_stats['arousal_range'][0]:.3f} to {summary_stats['arousal_range'][1]:.3f}")
print(f"Mean valence: {summary_stats['valence_mean']:.3f}")
print(f"Mean arousal: {summary_stats['arousal_mean']:.3f}")
print(f"Valence-Arousal correlation: {summary_stats['valence_arousal_correlation']:.3f}")

print("\nKEY RECOMMENDATIONS:")
print("1. Focus on tempo, spectral features, and energy for arousal prediction")
print("2. Prioritize spectral centroid and bandwidth for valence prediction")
print("3. Consider MFCC features for timbral characteristics")
print("4. Implement data augmentation to balance emotional quadrants")
print("5. Extract features from 30-second segments for consistent analysis")

# Save the processed dataset
output_path = '../data/processed/eda_sample_dataset.csv'
df.to_csv(output_path, index=False)
print(f"\nProcessed dataset saved to: {output_path}")